<a href="https://colab.research.google.com/github/rorocabezas/Detector_Neumonia_UDD/blob/main/api_rest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 0: Importa Librerias

In [ ]:
# Instalar flask
!pip install flask

# cargar librerias
from flask import Flask, request, render_template_string, redirect, url_for, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from werkzeug.utils import secure_filename
from google.colab import drive
from google.colab.output import eval_js
from IPython.display import display, Javascript
import numpy as np
import os

# Paso 1: Montar Google Drive


In [ ]:
# Montar Google Drive para acceder a los datos
drive.mount('/content/drive', force_remount=True)

# Cambiar al directorio del proyecto en Google Drive
os.chdir('/content/drive/MyDrive/UDD Bootcamps/Modulo 7/Proyecto_Final')

Mounted at /content/drive


# Paso 2: Cargar Modelo Entrenado

In [ ]:
# Cargar el modelo entrenado
try:
    model = load_model('best_model.keras')
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")

Modelo cargado exitosamente.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Paso 3: Configuracion y ejecución de Flask  

In [ ]:
# Configuración de Flask
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '/content/drive/MyDrive/UDD Bootcamps/Modulo 7/Proyecto_Final/uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Página principal con el formulario para subir la imagen de rayos X
@app.route('/')
def index():
    return render_template_string('''
        <!doctype html>
        <html lang="en">
          <head>
            <meta charset="utf-8">
            <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
            <title>Detector de Neumonía</title>
            <style>
              body {
                padding-top: 20px;
              }
              .container {
                max-width: 600px;
              }
            </style>
          </head>
          <body>
            <div class="container">
              <h1 class="mt-5 text-center">Detector de Neumonía</h1>
              <form action="/predict" method="post" enctype="multipart/form-data" class="mt-4">
                <div class="form-group">
                  <label for="file" class="font-weight-bold">Selecciona una imagen de rayos X:</label>
                  <input type="file" name="file" class="form-control-file" id="file" required>
                </div>
                <button type="submit" class="btn btn-primary btn-block">Enviar</button>
              </form>
            </div>
          </body>
        </html>
    ''')

# Ruta para manejar la predicción via POST
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        try:
            # Preprocesar la imagen
            img = image.load_img(file_path, target_size=(150, 150))
            img_array = image.img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            # Realizar la predicción y calcular el porcentaje de confianza
            prediction = model.predict(img_array)
            confidence = prediction[0][0] * 100  # Convertir a porcentaje
            result = 'Neumonía' if prediction[0] > 0.5 else 'Normal'

            # Verificar si la solicitud espera una respuesta JSON
            if request.headers.get('Accept') == 'application/json':
                return jsonify({
                    'result': result,
                    'confidence': f'{confidence:.2f}%'
                })

            # Devolver la plantilla HTML
            return render_template_string('''
                <!doctype html>
                <html lang="en">
                  <head>
                    <meta charset="utf-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
                    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
                    <title>Resultado</title>
                    <style>
                      body {
                        padding-top: 20px;
                      }
                      .container {
                        max-width: 600px;
                      }
                    </style>
                  </head>
                  <body>
                    <div class="container">
                      <h1 class="mt-5 text-center">Resultado de la Predicción</h1>
                      <div class="alert alert-{{ 'danger' if result == 'Neumonía' else 'success' }} mt-4" role="alert">
                        La imagen es clasificada como: <strong>{{ result }}</strong>
                      </div>
                      <div class="alert alert-info mt-4" role="alert">
                        Porcentaje de confianza: <strong>{{ confidence|round(2) }}%</strong>
                      </div>
                      <a href="/" class="btn btn-primary btn-block">Volver</a>
                    </div>
                  </body>
                </html>
            ''', result=result, confidence=confidence)

        except Exception as e:
            return jsonify({'error': str(e)}), 500

# Ejecutar la aplicación Flask
if __name__ == '__main__':
    PORT = 5000
    print(f"Eval JS...")
    eval_js(f"google.colab.kernel.proxyPort({PORT})")

    # Capturar y mostrar el enlace generado por eval_js
    display(Javascript('''
      (async () => {
        const url = await google.colab.kernel.proxyPort(5000);
        const output = document.createElement('div');
        output.innerHTML = `<p>Accede a la aplicación en esta url: <a href="${url}" target="_blank">${url}</a></p>`;
        document.querySelector('#output-area').appendChild(output);
      })();
    '''))

    app.run(host='0.0.0.0', port=PORT)

Eval JS...


<IPython.core.display.Javascript object>

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:20:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:21:05] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 854ms/step


INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:21:35] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:21:46] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:21:55] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:22:41] "GET / HTTP/1.1" 200 -
